# Michael comments
- if somebody joins relay, how much time is between joining and the next product they use? 
- - hypothesis is that they accidentally joined relay
- - what's the median time between product adoption? 

- Predictive modeling
- - start slow, (what we have is a markov chain), people move from one state to another. 
- - - what's the probability of moving from one state to another?
- - - markov chain is a good place to start (simplest model)
- - - then we could add more complexity (user features)

- Alternatively...
- - first to second product, or third product (ignore fourth, fifth, etc)
- Classification approach
- - what's the difference between users that have product 

- Don't worry about going back to the beginning and getting more data! (FOR NOW)


#### Questions we're answering
- what's the ideal onboarding flow?


## PROBLEMS WE NEED TO ADDRESS TO JUSTIFY METHODOLOGY:
- Users can take out multiple loans for varying purposes, which date does our "ORIGINATION_DATE" represent?
- Features change depending on what their first product is! Are there any features that are consistent across all products?




In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import itertools
from tqdm import tqdm

import plotly
# from chart_studio import plotly


In [2]:
dataframe = pd.read_csv('big_data.csv')

/var/folders/kc/jdw9593x3ksdmnhyd571zm440000gp/T/ipykernel_21422/1281625065.py:1: DtypeWarning: Columns (23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  dataframe = pd.read_csv('big_data.csv')


In [3]:
#create columns that show if a person uses a product
dataframe["uses_invest"] = ~dataframe['INV_OPEN_DATE'].isna() 
dataframe["uses_money"] = ~dataframe['MONEY_ACCOUNT_OPEN_DATE'].isna()
dataframe["uses_relay"] = ~dataframe['RELAY_ORIGINAL_DATE'].isna() 
dataframe["uses_loan"] = ~dataframe['ORIGINATION_DATE'].isna()
dataframe["uses_credit"] = ~dataframe['ONBOARDING_SUCCESS_DATE'].isna()

#create column that shows how many products a person uses
dataframe["num_products"] = (dataframe[['uses_invest', 'uses_money', 'uses_relay', 'uses_loan', 'uses_credit']] == True).sum(axis=1)

# create a column that shows how long a person has used a product. If they have never used a product, it will be NaN

dataframe['date_of_creating_new_columns'] = pd.to_datetime('today').date()

dataframe['INV_OPEN_DATE'] = pd.to_datetime(dataframe['INV_OPEN_DATE']).dt.date
dataframe['days_with_invest'] = (dataframe['date_of_creating_new_columns'] - dataframe['INV_OPEN_DATE']).dt.days

dataframe['MONEY_ACCOUNT_OPEN_DATE'] = pd.to_datetime(dataframe['MONEY_ACCOUNT_OPEN_DATE']).dt.date
dataframe['days_with_money'] = (dataframe['date_of_creating_new_columns'] - dataframe['MONEY_ACCOUNT_OPEN_DATE']).dt.days

dataframe['RELAY_ORIGINAL_DATE'] = pd.to_datetime(dataframe['RELAY_ORIGINAL_DATE']).dt.date
dataframe['days_with_relay'] = (dataframe['date_of_creating_new_columns'] - dataframe['RELAY_ORIGINAL_DATE']).dt.days

dataframe['ORIGINATION_DATE'] = pd.to_datetime(dataframe['ORIGINATION_DATE']).dt.date
dataframe['days_with_loan'] = (dataframe['date_of_creating_new_columns'] - dataframe['ORIGINATION_DATE']).dt.days

dataframe['ONBOARDING_SUCCESS_DATE'] = pd.to_datetime(dataframe['ONBOARDING_SUCCESS_DATE']).dt.date
dataframe['days_with_credit'] = (dataframe['date_of_creating_new_columns'] - dataframe['ONBOARDING_SUCCESS_DATE']).dt.days



In [4]:
#show all columns
pd.set_option('display.max_columns', None)
dataframe.head(40)

,ROW_NUM,USER_ID,INV_OPEN_DATE,INV_NUM_ACCOUNTS,INV_ACC_BAL,MONEY_ACCOUNT_OPEN_DATE,MONEY_ACCOUNT_STATUS,FICO,MONEY_DATA_UPDATE_DATE,MONEY_TOTAL_BALANCE,LAST_PAYMENT_DATE,ORIGINATION_DATE,PRINCIPAL,SL_ACTIVE_IND,PL_ACTIVE_IND,MORTGAGE_ACTIVE_IND,BANKING_ACTIVE_IND,INVEST_ACTIVE_IND,CC_ACTIVE_IND,CSM_ACTIVE_IND,ACTIVE,TOTAL_RECENCY_DAYS,RELAY_ORIGINAL_DATE,ONBOARDING_SUCCESS_DATE,FIRST_TRANSACTION_DATE,CREDIT_LINE,uses_invest,uses_money,uses_relay,uses_loan,uses_credit,num_products,date_of_creating_new_columns,days_with_invest,days_with_money,days_with_relay,days_with_loan,days_with_credit
0,1.0,16155391.0,2020-06-27,1.0,4532.07,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,True,False,False,False,False,1,2023-03-30,1006.0,NaN,NaN,NaN,NaN
1,1.0,16155395.0,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,False,True,True,236.0,2022-07-14,NaT,NaN,NaN,False,False,True,False,False,1,2023-03-30,NaN,NaN,259.0,NaN,NaN
2,1.0,16155396.0,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,False,True,True,208.0,2022-08-11,NaT,NaN,NaN,False,False,True,False,False,1,2023-03-30,NaN,NaN,231.0,NaN,NaN
3,1.0,16155403.0,2020-06-27,1.0,22.63,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,False,False,False,False,False,False,True,True,837.0,2020-11-20,NaT,NaN,NaN,True,False,True,False,False,2,2023-03-30,1006.0,NaN,860.0,NaN,NaN
4,1.0,16155404.0,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,False,True,True,101.0,2022-11-26,NaT,NaN,NaN,False,False,True,False,False,1,2023-03-30,NaN,NaN,124.0,NaN,NaN
5,1.0,16155408.0,2020-06-27,1.0,2534.93,2020-08-24,Active,512.0,2023-03-05,48.73,NaN,NaT,NaN,False,False,False,True,False,False,True,True,83.0,2022-12-14,NaT,NaN,NaN,True,True,True,False,False,3,2023-03-30,1006.0,948.0,106.0,NaN,NaN
6,1.0,16155416.0,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,False,True,True,199.0,2022-08-20,NaT,NaN,NaN,False,False,True,False,False,1,2023-03-30,NaN,NaN,222.0,NaN,NaN
7,1.0,16155418.0,2020-06-27,2.0,56677.76,2020-06-26,Active,536.0,2023-03-05,20.00,NaN,NaT,NaN,False,False,False,True,True,False,True,True,68.0,2022-12-29,NaT,NaN,NaN,True,True,True,False,False,3,2023-03-30,1006.0,1007.0,91.0,NaN,NaN
8,1.0,16155420.0,2022-06-02,1.0,50927.61,2021-08-30,Active,739.0,2023-03-05,282.95,2023-03-05,2021-08-31,36460.15,False,True,False,True,True,False,True,True,71.0,2022-12-26,NaT,NaN,NaN,True,True,True,True,False,4,2023-03-30,301.0,577.0,94.0,576.0,NaN
9,1.0,16155422.0,2020-06-27,1.0,22895.82,2020-06-26,Active,541.0,2023-03-05,5.00,NaN,NaT,NaN,False,False,False,True,True,False,True,True,590.0,2021-07-25,NaT,NaN,NaN,True,True,True,False,False,3,2023-03-30,1006.0,1007.0,613.0,NaN,NaN


### Columns for first and second product

In [5]:
# create a column 'first_product' that shows the first product a person used, and a column 'second_product' that shows the second product a person used. If they have never used a second product, it will be NaN

dataframe['first_product'] = np.nan
dataframe['second_product'] = np.nan

first_product_list = []
second_product_list = []

for i in tqdm(range(len(dataframe))):
    #create a list of the products that the user has
    products = []
    if dataframe.loc[i, 'uses_relay'] == True:
        products.append('relay')
    if dataframe.loc[i, 'uses_money'] == True:
        products.append('money')
    if dataframe.loc[i, 'uses_invest'] == True:
        products.append('invest')
    if dataframe.loc[i, 'uses_loan'] == True:
        products.append('loan')
    if dataframe.loc[i, 'uses_credit'] == True:
        products.append('credit')

    

    #sort the list of products by the number of days the user has used the product
    products.sort(key=lambda x: dataframe.loc[i, f'days_with_{x}'], reverse=True)

    #add the first two products to the list
    if len(products) >= 1:
        first_product_list.append(products[0])
    else:
        first_product_list.append(np.nan)
    if len(products) >= 2:
        second_product_list.append(products[1])
    else:
        second_product_list.append(np.nan)
        

dataframe['first_product'] = first_product_list
dataframe['second_product'] = second_product_list




    


100%|██████████| 3171463/3171463 [03:32<00:00, 14917.68it/s]


In [11]:
#save the dataframe
dataframe.to_csv('big_data_added_columns.csv', index=False)


In [30]:
#show how many users use personal loans
dataframe['uses_personal_loan'].value_counts()


False    3152547
True      165582
Name: uses_personal_loan, dtype: int64

# Understanding the xbuy journey


# Sankey diagram? More like JANKEY diagram amirite?

In [12]:
def convert_tuple_to_string(tup):
    str = ''
    for item in range(len(tup)):
        str = str + tup[item]
        if item < len(tup) - 1:
            str = str + '_'
    return str
    


In [22]:
products = ['relay', 'money', 'invest', 'loan', 'credit']

#permute the products
product_permutations_5 = list(itertools.permutations(products, 5))
product_permutations_4 = list(itertools.permutations(products, 4))
product_permutations_3 = list(itertools.permutations(products, 3))
product_permutations_2 = list(itertools.permutations(products, 2))
product_permutations_1 = list(itertools.permutations(products, 1))

product_permutations = product_permutations_5 + product_permutations_4 + product_permutations_3 + product_permutations_2 + product_permutations_1

#create a dataframe that will have 5 columns: lvl1, lvl2, lvl3, lvl4, and count
product_permutations_df = pd.DataFrame(columns=['lvl1', 'lvl2', 'lvl3', 'lvl4', 'lvl5', 'count'])
#fill the dataframe with the product permutations
for i in range(len(product_permutations)):
    for j in range(len(product_permutations[i])):
        product_permutations_df.loc[i, f'lvl{j+1}'] = convert_tuple_to_string(product_permutations[i][:j+1])

product_permutations_df['count'] = 0

product_permutations_df

,lvl1,lvl2,lvl3,lvl4,lvl5,count
0,relay,relay_money,relay_money_invest,relay_money_invest_loan,relay_money_invest_loan_credit,0
1,relay,relay_money,relay_money_invest,relay_money_invest_credit,relay_money_invest_credit_loan,0
2,relay,relay_money,relay_money_loan,relay_money_loan_invest,relay_money_loan_invest_credit,0
3,relay,relay_money,relay_money_loan,relay_money_loan_credit,relay_money_loan_credit_invest,0
4,relay,relay_money,relay_money_credit,relay_money_credit_invest,relay_money_credit_invest_loan,0
...,...,...,...,...,...,...
320,relay,NaN,NaN,NaN,NaN,0
321,money,NaN,NaN,NaN,NaN,0
322,invest,NaN,NaN,NaN,NaN,0
323,loan,NaN,NaN,NaN,NaN,0


In [23]:
#last populated column is the value held in the last column that has a value in it, with the options being lvl1, lvl2, lvl3, and lvl4
# product_permutations_df['last_populated_column'] = product_permutations_df['lvl4'].fillna(product_permutations_df['lvl3'].fillna(product_permutations_df['lvl2'].fillna(product_permutations_df['lvl1'])))
product_permutations_df['last_populated_column'] = product_permutations_df['lvl5'].fillna(product_permutations_df['lvl4'].fillna(product_permutations_df['lvl3'].fillna(product_permutations_df['lvl2'].fillna(product_permutations_df['lvl1']))))
last_populated_columns_dict = product_permutations_df.set_index('last_populated_column')['count'].to_dict()

In [24]:
last_populated_columns_dict

{'relay_money_invest_loan_credit': 0,
 'relay_money_invest_credit_loan': 0,
 'relay_money_loan_invest_credit': 0,
 'relay_money_loan_credit_invest': 0,
 'relay_money_credit_invest_loan': 0,
 'relay_money_credit_loan_invest': 0,
 'relay_invest_money_loan_credit': 0,
 'relay_invest_money_credit_loan': 0,
 'relay_invest_loan_money_credit': 0,
 'relay_invest_loan_credit_money': 0,
 'relay_invest_credit_money_loan': 0,
 'relay_invest_credit_loan_money': 0,
 'relay_loan_money_invest_credit': 0,
 'relay_loan_money_credit_invest': 0,
 'relay_loan_invest_money_credit': 0,
 'relay_loan_invest_credit_money': 0,
 'relay_loan_credit_money_invest': 0,
 'relay_loan_credit_invest_money': 0,
 'relay_credit_money_invest_loan': 0,
 'relay_credit_money_loan_invest': 0,
 'relay_credit_invest_money_loan': 0,
 'relay_credit_invest_loan_money': 0,
 'relay_credit_loan_money_invest': 0,
 'relay_credit_loan_invest_money': 0,
 'money_relay_invest_loan_credit': 0,
 'money_relay_invest_credit_loan': 0,
 'money_rela

In [25]:
#go through the dataframe and create the product permutations for each person
# the permutation is the order in which they used the products
# for example, a user with data like this:
# days_with_relay: 100
# days_with_money: NaN
# days_with_invest: 300
# days_with_loan: NaN
# would have a permutation of 'invest_relay' because they used invest first and then relay

for i in tqdm(range(len(dataframe))):
    #create a list of the products that the user has
    products = []
    if dataframe.loc[i, 'uses_relay'] == True:
        products.append('relay')
    if dataframe.loc[i, 'uses_money'] == True:
        products.append('money')
    if dataframe.loc[i, 'uses_invest'] == True:
        products.append('invest')
    if dataframe.loc[i, 'uses_loan'] == True:
        products.append('loan')
    if dataframe.loc[i, 'uses_credit'] == True:
        products.append('credit')
        
    #sort the list of products by the number of days the user has used the product
    products.sort(key=lambda x: dataframe.loc[i, f'days_with_{x}'])
    #convert the list of products to a string
    product_permutation = convert_tuple_to_string(tuple(products))
    #increment the count of the product permutation
    if len(products) > 0: 
        last_populated_columns_dict[product_permutation] += 1


100%|██████████| 3318129/3318129 [03:49<00:00, 14482.23it/s]


In [26]:
#populate the dataframe with the counts
for i in range(len(product_permutations_df)):
    product_permutations_df.loc[i, 'count'] = last_populated_columns_dict[product_permutations_df.loc[i, 'last_populated_column']]

#drop the last_populated_column column
product_permutations_df = product_permutations_df.drop(columns=['last_populated_column'])


In [27]:
product_permutations_df['lvl0'] = 'all'
#reindex the dataframe so the columns are in the correct order
product_permutations_df = product_permutations_df.reindex(columns=['lvl0', 'lvl1', 'lvl2', 'lvl3', 'lvl4', 'count'])

In [28]:
#save product_permutations_df to a csv file
product_permutations_df.to_csv('product_permutations.csv')


In [29]:
def genSankey(df,cat_cols=[],value_cols='',title='Sankey Diagram'):
    # maximum of 6 value cols -> 6 colors
    colorPalette = ['#4B8BBE','#306998','#FFE873','#FFD43B','#646464']
    labelList = []
    colorNumList = []
    for catCol in cat_cols:
        labelListTemp =  list(set(df[catCol].values))
        colorNumList.append(len(labelListTemp))
        labelList = labelList + labelListTemp
        
    # remove duplicates from labelList
    labelList = list(dict.fromkeys(labelList))
    
    # define colors based on number of levels
    colorList = []
    for idx, colorNum in enumerate(colorNumList):
        colorList = colorList + [colorPalette[idx]]*colorNum
        
    # transform df into a source-target pair
    for i in range(len(cat_cols)-1):
        if i==0:
            sourceTargetDf = df[[cat_cols[i],cat_cols[i+1],value_cols]]
            sourceTargetDf.columns = ['source','target','count']
        else:
            tempDf = df[[cat_cols[i],cat_cols[i+1],value_cols]]
            tempDf.columns = ['source','target','count']
            sourceTargetDf = pd.concat([sourceTargetDf,tempDf])
        sourceTargetDf = sourceTargetDf.groupby(['source','target']).agg({'count':'sum'}).reset_index()
        
    # add index for source-target pair
    sourceTargetDf['sourceID'] = sourceTargetDf['source'].apply(lambda x: labelList.index(x))
    sourceTargetDf['targetID'] = sourceTargetDf['target'].apply(lambda x: labelList.index(x))
    
    # creating the sankey diagram
    data = dict(
        type='sankey',
        node = dict(
          pad = 15,
          thickness = 20,
          line = dict(
            color = "black",
            width = 0.5
          ),
          label = labelList,
          color = colorList
        ),
        link = dict(
          source = sourceTargetDf['sourceID'],
          target = sourceTargetDf['targetID'],
          value = sourceTargetDf['count']
        )
      )
    
    layout =  dict(
        title = title,
        font = dict(
          size = 10
        )
    )
       
    fig = dict(data=[data], layout=layout)
    return fig

In [32]:
fig = genSankey(product_permutations_df,cat_cols=['lvl0','lvl1','lvl2','lvl3','lvl4'],value_cols='count',title='xbuy visualized')
plotly.offline.plot(fig, validate=False)

'temp-plot.html'

## "predicting xbuy into Personal Loans and into Money"

# Identify which loans are personal loans

In [28]:
#users that have personal_loan as their second product
print(f'users that have personal_loan as their second product: {len(dataframe[dataframe["second_product"] == "personal_loan"])}')
print(f'users that have money as their second product: {len(dataframe[dataframe["second_product"] == "money"])}')

users that have personal_loan as their second product: 10033
users that have money as their second product: 264122


In [29]:
dataframe[dataframe["second_product"] == "personal_loan"].head()

,ROW_NUM,USER_ID,INV_OPEN_DATE,INV_NUM_ACCOUNTS,INV_ACC_BAL,MONEY_ACCOUNT_OPEN_DATE,MONEY_ACCOUNT_STATUS,FICO,MONEY_DATA_UPDATE_DATE,MONEY_TOTAL_BALANCE,LAST_PAYMENT_DATE,ORIGINATION_DATE,PRINCIPAL,SL_ACTIVE_IND,PL_ACTIVE_IND,MORTGAGE_ACTIVE_IND,BANKING_ACTIVE_IND,INVEST_ACTIVE_IND,CC_ACTIVE_IND,CSM_ACTIVE_IND,ACTIVE,TOTAL_RECENCY_DAYS,RELAY_ORIGINAL_DATE,ONBOARDING_SUCCESS_DATE,FIRST_TRANSACTION_DATE,CREDIT_LINE,uses_invest,uses_money,uses_relay,uses_loan,uses_personal_loan,uses_credit,num_products,date_of_creating_new_columns,days_with_invest,days_with_money,days_with_relay,days_with_loan,days_with_credit,days_with_personal_loan,first_product,second_product
1057,1.0,19613239.0,2021-09-03,1.0,1007336.19,2022-01-19,Active,659.0,2023-02-21,1311.84,NaN,2023-01-25,15000.00,False,True,False,True,True,False,True,True,190.0,2022-08-16,NaT,NaN,NaN,True,True,True,True,True,False,4,2023-03-07,550.0,412.0,203.0,41.0,NaN,41.0,loan,personal_loan
3839,1.0,19625149.0,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,2023-02-19,7000.00,False,True,False,False,False,False,True,True,330.0,2022-03-29,2021-08-26,2022-05-21,15000.0,False,False,True,True,True,True,3,2023-03-07,NaN,NaN,343.0,16.0,558.0,16.0,loan,personal_loan
5414,1.0,19632836.0,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,2023-02-06,90000.00,False,True,False,False,False,False,True,True,187.0,2022-08-19,NaT,NaN,NaN,False,False,True,True,True,False,2,2023-03-07,NaN,NaN,200.0,29.0,NaN,29.0,loan,personal_loan
7099,1.0,19640681.0,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,2023-02-19,2023-01-19,12697.79,False,True,False,False,False,False,True,True,323.0,2022-04-05,NaT,NaN,NaN,False,False,True,True,True,False,2,2023-03-07,NaN,NaN,336.0,47.0,NaN,47.0,loan,personal_loan
7932,1.0,19642941.0,2021-08-10,2.0,8294569.01,2021-08-09,Active,779.0,2023-02-21,19978.38,2023-02-16,2023-02-07,10015.32,False,True,False,True,True,True,True,True,316.0,2022-04-12,2022-04-12,2022-04-15,12500.0,True,True,True,True,True,True,5,2023-03-07,574.0,575.0,329.0,28.0,329.0,28.0,loan,personal_loan
